In [ ]:
pip install duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 5.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import numpy.matlib
import pandas as pd
import duckdb

#from functools import wraps
#from typing import List, Optional, Tuple, cast
#import time
#import functools
#import pdb, sys

In [ ]:
sparse = pd.read_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/test01.csv')

In [ ]:
sparse_train = pd.read_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/sparse_train.csv')

In [ ]:
sparse_test = pd.read_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/sparse_test.csv')

In [ ]:
full_list=sparse[['Cust']]

In [ ]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter

        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update

        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose

    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost) / len(xi)


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)


    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])

    def out(self):
      return self._P, self._Q.T, self.get_complete_matrix()    


In [ ]:
train=duckdb.query("select * from full_list a left join sparse_train b on a.Cust=b.Cust").to_df()

In [ ]:
train.drop(['금융/보험서비스','기타상품'], axis=1, inplace=True)

In [ ]:
test=duckdb.query("select * from full_list a left join sparse_test b on a.Cust=b.Cust").to_df()

In [ ]:
sparse.drop(['금융/보험서비스','기타상품'], axis=1, inplace=True)

In [ ]:
sparse_zero = sparse.fillna(0)

In [ ]:
train_zero = train.fillna(0)

In [ ]:
test_zero = test.fillna(0)

In [ ]:
R = np.array(sparse_zero)
R_train = np.array(train_zero)
R_test = np.array(test_zero)

In [ ]:
# Not higm dim
R.shape

(26917, 58)

In [ ]:
R_train.shape

(26917, 58)

In [ ]:
R_test.shape

(26917, 58)

In [ ]:
R = R[:,1:]
R_train = R_train[:,2:]
R_test = R_test[:,2:]

In [ ]:
sparse_zero=sparse_zero.iloc[:,1:]

In [ ]:
train_zero=train_zero.iloc[:,2:]
test_zero=test_zero.iloc[:,2:]

In [ ]:
sparse_zero['sum']=sparse_zero.sum(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [ ]:
sparse_sum=sparse_zero.sort_values(by=['sum'], ascending=False)

ValueError: ignored

In [ ]:
def q4cut(s):
    return pd.qcut(s, 4, labels = ["G1","G2","G3","VIP"]).astype(str)

In [ ]:
sparse_sum['grade']=sparse_sum['sum'].transform(q4cut)

In [ ]:
sparse_zero['sum'].quantile(q=0.75, interpolation='nearest')

3322840.0

In [ ]:
sparse_sum

,Cust,가구,건강식품,건강용품,건해산물,계절가전,공구/안전용품,과일,과자,구기/필드스포츠,...,침구/수예,커피/차,컴퓨터,테넌트/음식점,패션잡화,퍼스널케어,헬스/피트니스,화장품/뷰티케어,sum,grade
9312,M343264688,18541000.0,3978000.0,238000.0,8980.0,0.0,54700.0,794380.0,80350.0,1871000.0,...,0.0,0.0,0.0,1520600.0,161516900.0,564000.0,0.0,5256000.0,386581410.0,VIP
10413,M384121563,0.0,0.0,59960.0,0.0,0.0,0.0,0.0,109300.0,605000.0,...,990000.0,423500.0,0.0,454500.0,286302050.0,1844800.0,0.0,42127000.0,379269980.0,VIP
7260,M268374518,0.0,281000.0,12390.0,0.0,0.0,0.0,104400.0,13130.0,0.0,...,0.0,40900.0,0.0,607400.0,358029000.0,0.0,0.0,1408900.0,371016180.0,VIP
1522,M055769504,0.0,17400.0,3000.0,0.0,0.0,0.0,0.0,10500.0,0.0,...,0.0,0.0,0.0,165100.0,365560000.0,0.0,0.0,1128200.0,369650800.0,VIP
7833,M288629527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8052000.0,0.0,0.0,25000.0,262697900.0,0.0,994200.0,998000.0,368955660.0,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14870,M549598435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,G1
18822,M698192014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,G1
4720,M173065733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,G1
11808,M438665188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,G1


In [ ]:
grade_VIP=duckdb.query("select * from sparse_sum where grade='VIP'").to_df()

In [ ]:
grade_G3=duckdb.query("select * from sparse_sum where grade='G3'").to_df()

In [ ]:
grade_G2=duckdb.query("select * from sparse_sum where grade='G2'").to_df()

In [ ]:
grade_G1=duckdb.query("select * from sparse_sum where grade='G1'").to_df()

In [ ]:
grade_VIP=grade_VIP.drop(['Cust','grade','sum'], axis=1)

In [ ]:
grade_G3=grade_G3.drop(['Cust','grade','sum'], axis=1)
grade_G2=grade_G2.drop(['Cust','grade','sum'], axis=1)
grade_G1=grade_G1.drop(['Cust','grade','sum'], axis=1)

In [ ]:
sparse_sum.to_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/list.csv', sep=',')

In [ ]:
# 단가 차이 있으니까 (가우시안) 표준화 or 정규화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler01 = MinMaxScaler()
scaler01.fit(sparse_zero)
scaler01_scaled = scaler01.transform(sparse_zero)

In [ ]:
scaler02 = MinMaxScaler()
scaler02.fit(train_zero)
scaler02_scaled = scaler02.transform(train_zero)

In [ ]:
scaler03 = MinMaxScaler()
scaler03.fit(test_zero)
scaler03_scaled = scaler03.transform(test_zero)

In [ ]:
scaler04 = MinMaxScaler()
scaler04.fit(grade_VIP)
VIP_scaled = scaler04.transform(grade_VIP)

In [ ]:
scaler05 = MinMaxScaler()
scaler05.fit(grade_G3)
G3_scaled = scaler05.transform(grade_G3)

In [ ]:
scaler06 = MinMaxScaler()
scaler06.fit(grade_G2)
G2_scaled = scaler06.transform(grade_G2)

In [ ]:
scaler07 = MinMaxScaler()
scaler07.fit(grade_G1)
G1_scaled = scaler07.transform(grade_G1)

In [ ]:
scaler01_scaled

In [ ]:
R=scaler01_scaled

In [ ]:
# run total data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_total = MatrixFactorization(R, k=3, learning_rate=0.1, reg_param=0.1, epochs=10, verbose=True)
    factorizer_total.fit()
    factorizer_total.print_results()

    sol_total=factorizer_total.out()


In [ ]:
# 3.799^-5 user latent, item latent, final R
user_latent01=sol_total[0]
item_latent01=sol_total[1]
finalR01=sol_total[2]

In [ ]:
R_train=scaler02_scaled

In [ ]:
R_test=scaler03_scaled

In [ ]:
# run train data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5) R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_train = MatrixFactorization(R_train, k=7, learning_rate=0.01, reg_param=0.01, epochs=10, verbose=True)
    factorizer_train.fit()
    factorizer_train.print_results()

    sol_train=factorizer_train.out()

In [ ]:
# user latent, item latent, final R
user_latent02=sol_train[0]
item_latent02=sol_train[1]
finalR02=sol_train[2]

In [ ]:
# run test data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5), R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_test = MatrixFactorization(R_test, k=7, learning_rate=0.01, reg_param=0.01, epochs=10, verbose=True)
    factorizer_test.fit()
    factorizer_test.print_results()

    sol_test=factorizer_test.out()

In [ ]:
# user latent, item latent, final R
user_latent03=sol_test[0]
item_latent03=sol_test[1]
finalR03=sol_test[2]

In [ ]:
# run test data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5), R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_VIP = MatrixFactorization(VIP_scaled, k=7, learning_rate=0.1, reg_param=0.1, epochs=10, verbose=True)
    factorizer_VIP.fit()
    factorizer_VIP.print_results()

    sol_VIP=factorizer_VIP.out()

In [ ]:
# user latent, item latent, final R
user_latent04=sol_VIP[0]
item_latent04=sol_VIP[1]
finalR04=sol_VIP[2]

In [ ]:
# run test data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5), R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_G1 = MatrixFactorization(G1_scaled, k=7, learning_rate=0.1, reg_param=0.1, epochs=10, verbose=True)
    factorizer_G1.fit()
    factorizer_G1.print_results()

    sol_G1=factorizer_G1.out()

Iteration: 10 ; cost = 0.0005
User Latent P:
[[ 0.01149178 -0.11023146  0.18375039 ... -0.16640561 -0.20172892
   0.05531564]
 [ 0.07303976  0.13993868  0.02558925 ...  0.03311001  0.1040784
   0.09655415]
 [ 0.04131132 -0.01483606  0.02278994 ...  0.02803702 -0.01272576
  -0.01483264]
 ...
 [-1.05813128  0.05169598  0.71189789 ... -0.22730851  1.80096325
   0.18754988]
 [ 0.3442648   0.54557358 -0.01389106 ...  0.28653066  0.77088233
   1.23200667]
 [-0.79988074 -1.49694324  0.8162607  ...  0.2150307  -0.03740765
   1.44568141]]
Item Latent Q:
[[ 4.92655462e-02 -8.42408840e-03  2.60039467e-03  1.68273063e-03
   3.24325465e-02 -3.89343265e-03  6.84240476e-04  2.34496255e-03
  -2.39143271e-02  7.69040697e-03  1.05553377e-03 -2.62394034e-03
   4.37292789e-02  4.00264697e-03 -4.06112526e-03 -1.75336190e-03
   2.57655675e-02 -2.65089405e-03 -1.88234834e-03  3.28624412e-03
  -8.02148777e-03  6.86194883e-03 -1.07760657e-03 -1.05028242e-02
  -8.67111215e-04  6.24409009e-04  1.01925510e-02 -2.

In [ ]:
# user latent, item latent, final R
user_latent05=sol_G1[0]
item_latent05=sol_G1[1]
finalR05=sol_G1[2]

In [ ]:
finalR01_ori = scaler01.inverse_transform(finalR01)

In [ ]:
finalR02_ori = scaler02.inverse_transform(finalR02)

In [ ]:
finalR03_ori = scaler03.inverse_transform(finalR03)

In [ ]:
finalR04_ori = scaler04.inverse_transform(finalR04)

In [ ]:
finalR05_ori = scaler05.inverse_transform(finalR05)

In [ ]:
finalR05_ori

array([[647260.11522387, 289307.95639787, 194088.61427272, ...,
        198027.39826268, 246755.97818331, 400034.97889679],
       [503680.72985127, 212705.34458299, 134732.09006786, ...,
        144388.25546513, 210790.23128022, 288092.9608795 ],
       [593767.12504991, 256636.88061543, 166542.32706289, ...,
        175390.35425379, 224588.73893153, 347974.10246745],
       ...,
       [ 55335.54075823,  56595.70628141,  17848.88899579, ...,
         39428.31109233, 158335.62773387, 130969.09201945],
       [ 87863.6709812 ,  49947.43973947,  33684.42735768, ...,
         60298.90658962, 228330.45931382,  99119.87008591],
       [-17563.97090832,  52201.40689924,  42727.4141256 , ...,
         73287.5815775 , 184197.01485864, 115545.0526961 ]])

In [ ]:
np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/finalR_VIP.csv", finalR04, delimiter=",")

In [ ]:
np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/finalR_G1.csv", finalR05, delimiter=",")

In [ ]:
finalR02.shape

(26917, 58)

In [ ]:
finalR01

array([[ 0.54282214,  1.06469488,  1.0324104 , ...,  0.20573438,
         0.55251728, -0.3004104 ],
       [ 0.75220865,  2.12631586,  1.05554061, ...,  2.00963895,
         2.50531184,  1.08045556],
       [ 1.75191192,  0.86801795,  2.09317728, ..., -0.00285413,
         1.36345465,  0.39958906],
       ...,
       [ 1.30359796,  0.7641471 ,  0.45467683, ..., -0.22473722,
         0.5168673 ,  0.03724268],
       [-0.20018991,  1.71999898,  2.60331259, ...,  0.13048368,
         0.4998987 , -0.19575099],
       [ 1.94914271,  0.19050381,  0.60571011, ...,  0.2230254 ,
         1.17765485, -0.35357657]])

In [ ]:
finalR02

array([[ 0.52131524,  1.36084301, -1.85691303, ...,  1.38742227,
         1.7817772 ,  0.21123787],
       [-1.14312432,  1.0671733 ,  2.65347199, ...,  1.50728064,
         0.325571  ,  0.44899263],
       [ 2.83410127,  0.74508679, -2.3813766 , ...,  0.86066426,
         2.03770363,  0.9160076 ],
       ...,
       [ 0.94962233,  1.3497357 ,  1.44432536, ...,  1.71624619,
         1.64445255, -0.04726747],
       [ 1.67455727,  0.37045275, -2.45297949, ...,  0.46441413,
         1.36713799,  0.62350476],
       [ 0.21137623,  0.51348283, -1.31555286, ...,  0.81790504,
         0.92466855,  0.35531855]])

In [ ]:
finalR03

array([[ 9.00894090e-01,  1.12683985e+00,  1.03152231e+00, ...,
        -3.58049283e-01,  1.85398529e+01,  4.69426397e-02],
       [-5.83622720e-01, -1.27685633e-01,  1.14489863e+01, ...,
        -7.06943115e-02,  9.99654089e+00,  2.86169728e-01],
       [-5.26122306e+00,  3.14294790e+00, -1.26067646e-01, ...,
         3.48521807e-01, -5.62524751e+00,  4.83558170e-01],
       ...,
       [-1.49829455e+00,  7.93736616e+00,  8.81607445e+00, ...,
         8.14629998e-03,  1.67312465e+01,  3.10354352e-01],
       [-1.41660256e+00,  5.98830694e+00,  7.21192503e+00, ...,
         7.05409569e-02,  1.11860923e+01,  2.72446637e-01],
       [-3.30712318e+00,  1.48285751e+00, -7.17312055e-01, ...,
         2.68769244e-01, -8.06879804e+00,  1.03788151e-01]])

In [ ]:
# Spectral clustering : 인간 / 카테고리 둘 다 해보자

In [ ]:
# affinity matrix : k by k